In [1]:
import tensorflow as tf
from tensorflow.python.ops import lookup_ops
from tensorflow.contrib import data

import os

In [2]:
OUT_DATA_PATH = '../data'
vocab_file = os.path.join(OUT_DATA_PATH, 'vocab.txt')

In [3]:
train_ctx = os.path.join(OUT_DATA_PATH, 'train.ctx')
train_utterance = os.path.join(OUT_DATA_PATH, 'train.utterance')
train_label = os.path.join(OUT_DATA_PATH, 'train.label')

In [4]:
vocab_table = lookup_ops.index_table_from_file(vocab_file, default_value=0)

In [5]:
ctx_dataset = data.TextLineDataset(train_ctx)
utterance_dataset = data.TextLineDataset(train_utterance)
label_dataset = data.TextLineDataset(train_label)

In [6]:
joint_dataset = data.Dataset.zip((ctx_dataset, utterance_dataset, label_dataset))

In [7]:
joint_dataset = joint_dataset.map(lambda ctx, utterance, label: (tf.string_split([ctx]).values, tf.string_split([utterance]).values, tf.string_to_number(label)))

In [8]:
joint_dataset = joint_dataset.map(lambda ctx, utterance, label: (vocab_table.lookup(ctx), vocab_table.lookup(utterance), label))

In [10]:
joint_dataset = joint_dataset.padded_batch(32, padded_shapes=(tf.TensorShape([None]), 
                                                          tf.TensorShape([None]), tf.TensorShape([])))

In [11]:
iterator = joint_dataset.make_initializable_iterator()

In [12]:
sess = tf.Session()

In [13]:
sess.run(tf.tables_initializer())

In [14]:
sess.run(iterator.initializer)

In [15]:
a = sess.run(iterator.get_next())